<font color='orange'>SKLEARN</font> - https://scikit-learn.org/stable/modules/semi_supervised.html

LAMDA-SSL - https://ygzwqzd.github.io/LAMDA-SSL/#/

# <font color='blue'>Wrapper methods</font>

## <font color='green'>Self-training</font><font color='orange'> SKLEARN</font>

## <font color='green'>Tri-training</font>, LAMDA-SSL

###  Semi-supervised classification -> Inductive -> Wrapper methods -> Co-training -> Single-view co-training -> <font color='green'>Tri-training</font>

### Opis

[A survey on semi-supervised learning]

Trzy klasyfikatory są naprzemiennie trenowane. Główna idea polega na tym, że gdy dwóch z trzech klasyfikatorów zgadza się co do prognozy dla danego punktu danych, przyjmuje się tę prognozę. Co ważne, "tri-training" <b>nie polega</b> na probabilistycznych prognozach indywidualnych klasyfikatorów. [A survey on semi-supervised learning]

In [15]:
from LAMDA_SSL.Algorithm.Classification.Tri_Training import Tri_Training
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from sklearn.svm import SVC
import numpy as np

file = open("Result/Tri_Training_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

SVM=SVC(C=1.0,kernel='linear',probability=True,gamma='auto')

model=Tri_Training(base_estimator=SVM,evaluation=evaluation,file=file,verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

print(result,file=file)

print(performance,file=file)

In [16]:
performance

{'accuracy': 0.9766081871345029,
 'precision': 0.9819819819819819,
 'Recall': 0.96875,
 'F1': 0.97469665581533,
 'AUC': 0.9970794392523364,
 'Confusion_matrix': array([[0.9375, 0.0625],
        [0.    , 1.    ]])}

### Własna implementacja

In [ ]:
import numpy as np
from sklearn.base import clone

class TriTraining:
    def __init__(self, classifier1, classifier2, classifier3):
        self.clf1 = classifier1
        self.clf2 = classifier2
        self.clf3 = classifier3

    def fit(self, labeled_data, labeled_labels, unlabeled_data):
        # Początkowe dopasowanie klasyfikatorów do etykietowanych danych
        self.clf1.fit(labeled_data, labeled_labels)
        self.clf2.fit(labeled_data, labeled_labels)
        self.clf3.fit(labeled_data, labeled_labels)

        # Pętla uczenia klasyfikatorów na podstawie danych pseudoetykietowanych
        while unlabeled_data.shape[0] > 0:
            # Prognozy klasyfikatorów dla nieoznaczonych danych
            pred1 = self.clf1.predict(unlabeled_data)
            pred2 = self.clf2.predict(unlabeled_data)
            pred3 = self.clf3.predict(unlabeled_data)

            # Znajdź indeksy, gdzie dwóch klasyfikatorów zgadza się co do prognozy
            idx12, = np.where(pred1 == pred2)
            idx13, = np.where(pred1 == pred3)
            idx23, = np.where(pred2 == pred3)

            # Ucz klasyfikatory na danych pseudoetykietowanych
            if len(idx12) > 0:
                self.clf3.fit(unlabeled_data[idx12], pred1[idx12])
            if len(idx13) > 0:
                self.clf2.fit(unlabeled_data[idx13], pred1[idx13])
            if len(idx23) > 0:
                self.clf1.fit(unlabeled_data[idx23], pred2[idx23])

            # Usuń użyte dane z nieoznaczonych danych
            unlabeled_data = np.delete(unlabeled_data, np.concatenate([idx12, idx13, idx23]), axis=0)

    def predict(self, data):
        pred1 = self.clf1.predict(data)
        pred2 = self.clf2.predict(data)
        pred3 = self.clf3.predict(data)

        # Głosowanie większościowe dla prognozy
        final_pred = []
        for p1, p2, p3 in zip(pred1, pred2, pred3):
            if p1 == p2 or p1 == p3:
                final_pred.append(p1)
            else:
                final_pred.append(p2)
        return np.array(final_pred)


##  <font color='green'>ASSEMBLE</font> LAMDA-SSL

###  Semi-supervised classification -> Inductive -> Wrapper methods -> Boosting -> <font color='green'>ASSEMBLE</font>

### Opis

[A survey on semi-supervised learning]

Pseudo-etykietowanie: W algorytmie "ASSEMBLE", dane, które początkowo nie mają etykiety, są etykietowane (pseudo-etykietowane) przez model w każdej iteracji. To pozwala na wykorzystanie tych punktów danych do trenowania klasyfikatora w kolejnych iteracjach.

- Adaptacyjna konstrukcja klasyfikatora: Te pseudo-etykietowane punkty danych są następnie wykorzystywane do budowy lub dostosowywania klasyfikatora. Dzięki temu klasyfikator może stopniowo uczyć się z coraz większej ilości danych, co może prowadzić do lepszej generalizacji.

- Maksymalizacja marginesu klasyfikacji: Jednym z głównych celów "ASSEMBLE" jest maksymalizacja marginesu klasyfikacji w przestrzeni funkcji. Margines klasyfikacji to odległość między punktami danych a granicą decyzyjną klasyfikatora. Maksymalizując ten margines, algorytm dąży do lepszego rozdzielania klas.

- Wykorzystanie bootstrapingu: Problemem w pseudo-etykietowaniu jest to, że nie jest pewne, które punkty danych powinny być wykorzystywane w następnej iteracji uczenia. Aby rozwiązać ten problem, "ASSEMBLE" korzysta z techniki bootstrapingu. Oznacza to, że próbkuje się równomiernie i losowo z etykietowanych i pseudo-etykietowanych danych, pozwalając modelowi uczyć się z różnorodnego zestawu danych.

- Pomysł stojący za "ASSEMBLE" i innymi technikami uczenia półnadzorowanego polega na wykorzystaniu niewykorzystanych informacji zawartych w nieoznaczonych danych. Pseudo-etykietowanie, choć niosące pewne ryzyko wprowadzenia błędnych etykiet, może znacznie zwiększyć ilość dostępnych danych do uczenia, co jest szczególnie przydatne w sytuacjach, gdy oznaczone dane są trudne lub kosztowne do uzyskania.

In [3]:
from LAMDA_SSL.Algorithm.Classification.Assemble import Assemble
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from sklearn.svm import SVC
import numpy as np

file = open("Result/Assemble_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

# Base estimater
SVM=SVC(probability=True)

model=Assemble(T=100,base_estimator=SVM,evaluation=evaluation,verbose=True,file=file)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

{'accuracy': 0.9766081871345029,
 'precision': 0.9819819819819819,
 'Recall': 0.96875,
 'F1': 0.97469665581533,
 'AUC': 0.9978095794392523,
 'Confusion_matrix': array([[0.9375, 0.0625],
        [0.    , 1.    ]])}

### Własna implementacja

In [1]:
import numpy as np
from sklearn.base import clone
from sklearn.utils import resample

class ASSEMBLE:
"""
    Implementacja metody ASSEMBLE do uczenia pół-nadzorowanego.

    Podstawy ASSEMBLE:
    -----------------
    Metoda `ASSEMBLE` jest strategią uczenia pół-nadzorowanego, w której klasyfikator bazowy jest szkolony wielokrotnie,
    przy czym w każdej iteracji dane nieoznaczone są etykietowane (pseudo-labeling) i łączone z rzeczywistymi danymi etykietowanymi.
    Główna idea polega na tym, żeby wykorzystać moc klasyfikatora bazowego do "pseudo-etykietowania" danych nieoznaczonych i 
    następnie trenować klasyfikator na kombinacji danych oznaczonych i pseudo-etykietowanych.

    Metoda `fit`:
    ------------
    1. Łączenie danych: Dane oznaczone i nieoznaczone są łączone w jeden zestaw danych (`X_combined`).
    2. Pętla przez iteracje: W każdej iteracji model jest trenowany na danych oznaczonych, a następnie jest używany do przewidywania
       etykiet dla danych nieoznaczonych (`pseudo_labels`).
    3. Bootstrap: Następnie tworzony jest zestaw danych bootstrap poprzez losowe próbkowanie (z powtórzeniami) z połączonych 
       danych oznaczonych i pseudo-etykietowanych.
    4. Podział na oznaczone i nieoznaczone: Po stworzeniu zestawu bootstrap, dzielimy go z powrotem na dane oznaczone i 
       nieoznaczone na potrzeby następnej iteracji.

    Metoda `predict`:
    ----------------
    Po zakończeniu procesu uczenia się mamy wiele nauczonych modeli. W metodzie `predict` zbieramy przewidywania z każdego modelu 
    dla danego zestawu danych X. Następnie, dla każdego punktu danych, decydujemy o końcowej etykiecie na podstawie głosowania 
    większościowego spośród wszystkich modeli.
"""
    def __init__(self, base_classifier, n_iterations=10):
        self.base_classifier = base_classifier
        self.n_iterations = n_iterations
        self.models = []
    
    def fit(self, X_labeled, y_labeled, X_unlabeled):
        X_combined = np.vstack((X_labeled, X_unlabeled))
        
        for _ in range(self.n_iterations):
            model = clone(self.base_classifier)
            model.fit(X_labeled, y_labeled)
            self.models.append(model)
            
            pseudo_labels = model.predict(X_unlabeled)
            
            X_bootstrap, y_bootstrap = resample(X_combined, np.hstack((y_labeled, pseudo_labels)))
            
            X_labeled, X_unlabeled = X_bootstrap[:len(X_labeled)], X_bootstrap[len(X_labeled):]
            y_labeled = y_bootstrap[:len(y_labeled)]
            
    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

# Example usage:
# from sklearn.datasets import load_iris
# from sklearn.tree import DecisionTreeClassifier

# data = load_iris()
# X_labeled, y_labeled = data.data[:30], data.target[:30]
# X_unlabeled = data.data[30:]

# model = ASSEMBLE(DecisionTreeClassifier(), n_iterations=5)
# model.fit(X_labeled, y_labeled, X_unlabeled)
# predictions = model.predict(data.data[30:])


In [36]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Wczytanie danych
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.6, random_state=42)

# Dzielenie zbioru treningowego na dane oznaczone i nieoznaczone
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X_train, y_train, test_size=0.6, random_state=42)

# Trening i testowanie zwykłego DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_labeled, y_labeled)
predictions_tree = clf.predict(X_test)
accuracy_tree = accuracy_score(y_test, predictions_tree)
print(f"Accuracy of DecisionTreeClassifier: {accuracy_tree:.2f}")

# Trening i testowanie ASSEMBLE DecisionTreeClassifier
assemble_model = ASSEMBLE(DecisionTreeClassifier(), n_iterations=5)
assemble_model.fit(X_labeled, y_labeled, X_unlabeled)
predictions_assemble = assemble_model.predict(X_test)
accuracy_assemble = accuracy_score(y_test, predictions_assemble)
print(f"Accuracy of ASSEMBLE DecisionTreeClassifier: {accuracy_assemble:.2f}")


Accuracy of DecisionTreeClassifier: 0.97
Accuracy of ASSEMBLE DecisionTreeClassifier: 0.73


##  <font color='green'>Semiboost</font> LAMDA-SSL

###  Semi-supervised classification -> Inductive -> Wrapper methods -> Boosting -> <font color='green'>Semiboost</font>

### Opis

[A survey on semi-supervised learning]

Algorytm uczenia pół-nadzorowanego o nazwie SemiBoost rozwiązuje problem wyboru danych używanych przez klasyfikatory bazowe, opierając się na założeniu rozmieszczenia punktów na pewnej rozmaitości (manifold assumption) i korzystając z metod opartych na grafach. Dla każdego punktu z nieoznakowanych danych przydzielana jest pseudo-etykieta, a następnie obliczane jest zaufanie do tego przypisania. Na podstawie grafu, który koduje podobieństwo między danymi, pewien podzbiór tych pseudo-etykietowanych danych jest dodawany do zbioru etykietowanych danych dla kolejnego klasyfikatora. SemiBoost był skutecznie stosowany, między innymi, do śledzenia obiektów na wideo.

SemiBoost wykorzystuje standardowy model klasyfikacji z boostingu, gdzie końcowa etykieta jest liniową kombinacją przewidywań poszczególnych uczących się. W kontekście SemiBoost, sukces znaczenia testowego powinien spełniać trzy wymagania:

- Przewidywane etykiety dla nieoznakowanych danych powinny być spójne dla punktów blisko siebie.
- Przewidywane etykiety dla nieoznakowanych danych powinny być spójne z etykietami pobliskich oznakowanych danych.
- Przewidywane etykiety dla oznakowanych danych powinny odpowiadać ich prawdziwym etykietom.

Te wymagania są przedstawione w postaci problemu optymalizacji z ograniczeniami. SemiBoost używa boostingu do rozwiązania tego problemu optymalizacji, gdzie dwie pierwsze cechy są uchwycone przez funkcję celu, a ostatnia jest narzucana jako ograniczenie. Wzór matematyczny przedstawia koszty wynikające z niespójności między nieoznakowanymi i łączonymi etykietowanymi oraz nieoznakowanymi danymi.


----------------------------
Streszczenie

SemiBoost to algorytm semi-nadzorowanego uczenia się, który łączy techniki boostingu z metodami opartymi na grafach. Oto kilka kluczowych koncepcji dotyczących SemiBoost:

1. Manifold Assumption: SemiBoost opiera się na tzw. założeniu manifoldowym. To założenie mówi, że podobne dane wejściowe mają tendencję do otrzymywania podobnych etykiet. Innymi słowy, jeśli dwa punkty danych są blisko siebie w przestrzeni, prawdopodobnie należą do tej samej klasy. W kontekście SemiBoost i wielu innych algorytmów semi-nadzorowanego uczenia się, kernel RBF jest używany do mierzenia podobieństwa (lub odległości) między punktami danych.

2. Pseudo-etykietowanie: Każdy nieoznakowany punkt danych otrzymuje pseudo-etykietę, a następnie obliczane jest zaufanie do tego przydziału. Na podstawie tej pewności oraz struktury manifoldowej danej części danych, wybrane są punkty danych do dalszego trenowania.
3. Boosting: Podobnie jak w standardowych algorytmach boostingu, SemiBoost tworzy liniową kombinację klasyfikatorów bazowych, z których każdy skupia się na trudnych do klasyfikacji przykładach z poprzednich iteracji.
4. Optymalizacja z ograniczeniami: Kluczowym aspektem SemiBoost jest postawienie problemu jako optymalizacji z ograniczeniami. Algorytm stara się minimalizować niespójność etykiet między danymi etykietowanymi i nieoznakowanymi, jednocześnie zapewniając, że przewidywane etykiety dla danych etykietowanych odpowiadają ich prawdziwym etykietom.
5. Metody oparte na grafach: SemiBoost korzysta z technik grafowych do określenia podobieństwa między punktami danych. Wykorzystuje to do zapewnienia, że etykiety są spójne dla punktów danych leżących na tym samym manifoldzie.

Zastosowania: Choć algorytm SemiBoost można stosować w różnych dziedzinach, był on szczególnie skuteczny w zadaniach takich jak śledzenie obiektów w sekwencjach wideo, co pokazuje jego zdolność do wykorzystywania zarówno etykietowanych, jak i nieoznakowanych danych w trudnych warunkach.

Podsumowując, SemiBoost to zaawansowany algorytm semi-nadzorowanego uczenia się, który łączy moc technik boostingu z zaawansowanymi technikami grafowymi w celu poprawy wydajności w zadaniach klasyfikacji.






In [4]:
from LAMDA_SSL.Algorithm.Classification.SemiBoost import SemiBoost
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from sklearn.svm import SVC
import numpy as np

file = open("Result/SemiBoost_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

SVM=SVC(C=1.0,kernel='rbf',probability=True,gamma='auto')

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

model=SemiBoost(gamma=10,base_estimator=SVM,evaluation=evaluation,file=file,verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

{'accuracy': 0.9473684210526315,
 'precision': 0.9518008474576272,
 'Recall': 0.9359667056074766,
 'F1': 0.9428666889408619,
 'AUC': 0.9929906542056075,
 'Confusion_matrix': array([[0.890625  , 0.109375  ],
        [0.01869159, 0.98130841]])}

# <font color='blue'>Unsupervised preprocessing</font>

brak

# <font color='blue'>Intrinsically semi-supervised</font>

## <font color='green'>LapSVM</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Manifolds -> Manifold regularization -> <font color='green'>LapSVM</font>

### Opis

[Dokumentacja LAMDA-SSL]

Bazuje ona na założeniu manifoldu. Klasyczne SVM oparte na oryginalnej przestrzeni cech pomija kluczowe cechy próbek. LapSVM dodaje termin regularyzacji manifoldu do celu optymalizacyjnego SVM, aby nauczyć się istotnego rozkładu próbek. LapSVM buduje model grafu wykorzystując wszystkie próbki, uzyskuje macierz wag modelu grafu przez podobieństwo cech próbek i oblicza jego macierz Laplace'a. Ten termin regularyzacji prowadzi do spójności przewidywanych wyników dla sąsiadujących próbek w grafie. W przeciwieństwie do TSVM, LapSVM kara tylko błędne klasyfikacje próbek oznaczonych, ale wykorzystuje wszystkie próbki podczas budowy modelu grafu, dzięki czemu nieoznaczone próbki uczestniczą w procesie uczenia poprzez rozkład próbek na rozmaitości.

In [6]:
from LAMDA_SSL.Algorithm.Classification.LapSVM import LapSVM
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
import numpy as np

file = open("Result/LapSVM_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

model=LapSVM(neighbor_mode='connectivity',
           gamma_d=0.03,
           n_neighbor= 5,
           gamma_k=0.03,
           gamma_A= 0.03,
           gamma_I= 0,
           evaluation=evaluation,file=file,verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

{'accuracy': 0.9649122807017544,
 'precision': 0.9691441441441442,
 'Recall': 0.9562646028037383,
 'F1': 0.962044983722995,
 'AUC': 0.9944509345794392,
 'Confusion_matrix': array([[0.921875  , 0.078125  ],
        [0.00934579, 0.99065421]])}

## <font color='green'>Ladder networks</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Perturbation-based -> <font color='green'>Ladder networks</font>

### Opis

[LAMDA-SSL]
- Zaproponowany przez Rasmusa i innych.
- Struktura bazująca na autoenkoderze, gdzie wyjścia z ostatniej warstwy enkodera są traktowane jako etykiety miękkie.
- Sieć przyjmuje dwa metody kodowania: klasyczne kodowanie bez szumu oraz kodowanie z szumem, gdzie szum jest dodawany do wejść każdej warstwy.
- Pierwszym krokiem jest kodowanie z szumem i bez szumu próbek. Następnie uzyskuje się reprezentacje zaszumione i niezaszumione każdej warstwy.
- Dekoder jest używany do dekodowania wyniku kodowania z szumem, uzyskując zaszumione reprezentacje dekodowania dla każdej warstwy.
- Wykorzystywana jest funkcja straty błędu średniokwadratowego (MSE) do obliczania niespójności między reprezentacją kodowaną bez szumu a reprezentacją dekodowaną z szumem na każdym poziomie. To obejmuje również oryginalne dane wejściowe jako warstwę zerową.
- Wstępnie określone wagi są używane do określenia wag niespójności dla każdej warstwy.
- Niespójności są ważone hierarchicznie w funkcji straty bez nadzoru, co zwiększa odporność modelu.
- Regularyzacja spójności w Ladder Network używa zaszumionej reprezentacji kodowanej jako "most" do penalizacji niespójności między reprezentacją kodowaną bez szumu a zaszumioną reprezentacją dekodowaną.
- Dzięki temu autoenkoder może uzyskać spójne reprezentacje enkodera i dekodera na wszystkich poziomach. Ponadto reprezentacje warstwy ukrytej pozostają spójne, niezależnie od tego, czy dodany jest szum.
- Dzięki temu model jest odporny na zakłócenia.

In [13]:
from LAMDA_SSL.Algorithm.Classification.LadderNetwork import LadderNetwork
from LAMDA_SSL.Opitimizer.Adam import Adam
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset
from LAMDA_SSL.Scheduler.LinearWarmup import LinearWarmup
from LAMDA_SSL.Dataset.Vision.Mnist import Mnist
import torch.nn as nn

dataset=Mnist(root='Download\mnist',labeled_size=6000,stratified=True,shuffle=True,download=False,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y

unlabeled_X=dataset.unlabeled_X

test_X=dataset.test_X
test_y=dataset.test_y

valid_X=dataset.valid_X
valid_y=dataset.valid_y

labeled_dataset=LabeledDataset(pre_transform=dataset.pre_transform,transforms=dataset.transforms,
                               transform=dataset.transform,target_transform=dataset.target_transform)
unlabeled_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.unlabeled_transform)
valid_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.valid_transform)
test_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.test_transform)

#dataloader
labeled_dataloader=LabeledDataLoader(batch_size=100,num_workers=0,drop_last=True)
unlabeled_dataloader=UnlabeledDataLoader(num_workers=0,drop_last=True)
valid_dataloader=UnlabeledDataLoader(batch_size=100,num_workers=0,drop_last=False)
test_dataloader=UnlabeledDataLoader(batch_size=100,num_workers=0,drop_last=False)

# sampler
labeled_sampler=RandomSampler(replacement=True,num_samples=100*540)
unlabeled_sampler=RandomSampler(replacement=False)
test_sampler=SequentialSampler()
valid_sampler=SequentialSampler()

# optimizer
optimizer=Adam(lr=0.02)

# scheduler
scheduler=LinearWarmup(num_warmup_steps=15,num_training_steps=10,verbose=False)

# evalutation
evaluation={
    'accuracy':Accuracy(),
    'top_5_accuracy':Top_k_Accurary(k=5),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

file = open("Result/LadderNetwork_MNIST.txt", "w")

model=LadderNetwork(noise_std=0.2,
                     lambda_u=[0.1, 0.1, 0.1, 0.1, 0.1, 10., 1000.],
                     dim_encoder=[1000, 500, 250, 250, 250],
                     encoder_activations=[nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()],
                     mu=1,weight_decay=5e-4,
                     epoch=2,num_it_epoch=10,num_it_total=540*10,eval_epoch=1,
                     optimizer=optimizer,scheduler=scheduler,evaluation=evaluation,device='cpu',
                     labeled_dataset=labeled_dataset, unlabeled_dataset=unlabeled_dataset, valid_dataset=valid_dataset,
                     test_dataset=test_dataset,
                     labeled_sampler=labeled_sampler, unlabeled_sampler=unlabeled_sampler, valid_sampler=valid_sampler,
                     test_sampler=test_sampler,
                     labeled_dataloader=labeled_dataloader, unlabeled_dataloader=unlabeled_dataloader,
                     valid_dataloader=valid_dataloader, test_dataloader=test_dataloader,
                     file=None,verbose=False)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X,valid_X=valid_X,valid_y=valid_y)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

{'accuracy': 0.8563,
 'top_5_accuracy': 0.9683,
 'precision': 0.8668545707173398,
 'Recall': 0.8545707723710525,
 'F1': 0.8534983564223332,
 'AUC': 0.9823251238971753,
 'Confusion_matrix': array([[9.69387755e-01, 0.00000000e+00, 0.00000000e+00, 4.08163265e-03,
         0.00000000e+00, 2.04081633e-03, 1.63265306e-02, 1.02040816e-03,
         4.08163265e-03, 3.06122449e-03],
        [0.00000000e+00, 9.87665198e-01, 0.00000000e+00, 4.40528634e-03,
         0.00000000e+00, 8.81057269e-04, 5.28634361e-03, 0.00000000e+00,
         1.76211454e-03, 0.00000000e+00],
        [6.39534884e-02, 3.48837209e-02, 7.43217054e-01, 4.94186047e-02,
         8.72093023e-03, 2.90697674e-03, 2.42248062e-02, 2.32558140e-02,
         2.32558140e-02, 2.61627907e-02],
        [2.57425743e-02, 6.93069307e-03, 3.96039604e-03, 8.48514851e-01,
         0.00000000e+00, 5.14851485e-02, 5.94059406e-03, 1.98019802e-02,
         1.68316832e-02, 2.07920792e-02],
        [8.14663951e-03, 2.03665988e-02, 1.01832994e-03, 0.0

## <font color='green'>$\Pi$-model</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Perturbation-based -> <font color='green'>$\Pi$-model</font>

### Opis

In [14]:
from LAMDA_SSL.Augmentation.Vision.RandomHorizontalFlip import RandomHorizontalFlip
from LAMDA_SSL.Augmentation.Vision.RandomCrop import RandomCrop
from LAMDA_SSL.Dataset.Vision.CIFAR10 import CIFAR10
from LAMDA_SSL.Opitimizer.SGD import SGD
from LAMDA_SSL.Scheduler.CosineAnnealingLR import CosineAnnealingLR
from LAMDA_SSL.Network.WideResNet import WideResNet
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader
from LAMDA_SSL.Algorithm.Classification.PiModel import PiModel
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler
from sklearn.pipeline import Pipeline
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset

# dataset
dataset=CIFAR10(root='Download\cifar-10-python',labeled_size=4000,stratified=True,shuffle=True,download=False,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y

unlabeled_X=dataset.unlabeled_X

test_X=dataset.test_X
test_y=dataset.test_y

valid_X=dataset.valid_X
valid_y=dataset.valid_y

labeled_dataset=LabeledDataset(pre_transform=dataset.pre_transform,transforms=dataset.transforms,
                               transform=dataset.transform,target_transform=dataset.target_transform)

unlabeled_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.unlabeled_transform)

valid_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.valid_transform)

test_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.test_transform)

# sampler
labeled_sampler=RandomSampler(replacement=True,num_samples=64*(2**20))
unlabeled_sampler=RandomSampler(replacement=True)
valid_sampler=SequentialSampler()
test_sampler=SequentialSampler()

#dataloader
labeled_dataloader=LabeledDataLoader(batch_size=64,num_workers=0,drop_last=True)
unlabeled_dataloader=UnlabeledDataLoader(num_workers=0,drop_last=True)
valid_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)
test_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)

# augmentation

augmentation=Pipeline([('RandomHorizontalFlip',RandomHorizontalFlip()),
                        ('RandomCrop',RandomCrop(padding=0.125,padding_mode='reflect')),
                      ])

# optimizer
optimizer=SGD(lr=0.03,momentum=0.9,nesterov=True)

# scheduler
scheduler=CosineAnnealingLR(eta_min=0,T_max=2**20)

# network
network=WideResNet(num_classes=10,depth=28,widen_factor=2,drop_rate=0)

evaluation={
    'accuracy':Accuracy(),
    'top_5_accuracy':Top_k_Accurary(k=5),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

file = open("Result/PiModel_CIFAR10.txt", "w")

model=PiModel(lambda_u=10,warmup=0.4,mu=1,weight_decay=5e-4,ema_decay=0.999,
               epoch=1,num_it_epoch=10,num_it_total=540*10,
               eval_it=2000,device='cpu',
               labeled_dataset=labeled_dataset,
               unlabeled_dataset=unlabeled_dataset,
               valid_dataset=valid_dataset,
               test_dataset=test_dataset,
               labeled_sampler=labeled_sampler,
               unlabeled_sampler=unlabeled_sampler,
               valid_sampler=valid_sampler,
               test_sampler=test_sampler,
               labeled_dataloader=labeled_dataloader,
               unlabeled_dataloader=unlabeled_dataloader,
               valid_dataloader=valid_dataloader,
               test_dataloader=test_dataloader,
               augmentation=augmentation,
               network=network,
               optimizer=optimizer,
               scheduler=scheduler,
               evaluation=evaluation,
               file=file,verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X,valid_X=valid_X,valid_y=valid_y)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting Download\cifar-10-python\cifar-10-python.tar.gz to Download\cifar-10-python
1
2
3
4
5
6
7
8
9
10
accuracy   0.1003
top_5_accuracy   0.5001
precision   0.076775956284153
Recall   0.10029999999999999
F1   0.0191558241070029


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.5031681
Confusion_matrix   [[0.    0.    0.    0.    0.    0.    0.075 0.    0.001 0.924]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.021 0.    0.    0.979]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]
 [0.    0.    0.    0.    0.    0.    0.016 0.    0.004 0.98 ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.001 0.999]]


KeyboardInterrupt: 

## <font color='green'>Temporal ensembling</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Perturbation-based -> <font color='green'>Temporal ensembling</font>

### Opis

In [15]:
from LAMDA_SSL.Augmentation.Vision.RandomHorizontalFlip import RandomHorizontalFlip
from LAMDA_SSL.Augmentation.Vision.RandomCrop import RandomCrop
from LAMDA_SSL.Dataset.Vision.CIFAR10 import CIFAR10
from LAMDA_SSL.Opitimizer.SGD import SGD
from LAMDA_SSL.Scheduler.CosineAnnealingLR import CosineAnnealingLR
from LAMDA_SSL.Network.WideResNet import WideResNet
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader
from LAMDA_SSL.Algorithm.Classification.TemporalEnsembling import TemporalEnsembling
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler
from sklearn.pipeline import Pipeline
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset

# dataset
dataset=CIFAR10(root='Download\cifar-10-python',labeled_size=4000,stratified=True,shuffle=True,download=False,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y

unlabeled_X=dataset.unlabeled_X

test_X=dataset.test_X
test_y=dataset.test_y

valid_X=dataset.valid_X
valid_y=dataset.valid_y

labeled_dataset=LabeledDataset(pre_transform=dataset.pre_transform,transforms=dataset.transforms,
                               transform=dataset.transform,target_transform=dataset.target_transform)

unlabeled_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.unlabeled_transform)

valid_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.valid_transform)

test_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.test_transform)

# sampler
labeled_sampler=RandomSampler(replacement=True,num_samples=460*100)
unlabeled_sampler=RandomSampler(replacement=False)
valid_sampler=SequentialSampler()
test_sampler=SequentialSampler()

#dataloader
labeled_dataloader=LabeledDataLoader(batch_size=100,num_workers=0,drop_last=True)
unlabeled_dataloader=UnlabeledDataLoader(num_workers=0,drop_last=True)
valid_dataloader=UnlabeledDataLoader(batch_size=100,num_workers=0,drop_last=False)
test_dataloader=UnlabeledDataLoader(batch_size=100,num_workers=0,drop_last=False)

# augmentation
augmentation=Pipeline([('RandomHorizontalFlip',RandomHorizontalFlip()),
                        ('RandomCrop',RandomCrop(padding=0.125,padding_mode='reflect')),
                    ])

# optimizer
optimizer=SGD(lr=0.1,momentum=0.9,weight_decay=5e-4)

# scheduler
scheduler=CosineAnnealingLR(eta_min=0.0001,T_max=400)

# network
network=WideResNet(num_classes=10,depth=28,widen_factor=2,drop_rate=0)

# evalutation
evaluation={
    'accuracy':Accuracy(),
    'top_5_accuracy':Top_k_Accurary(k=5),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

file = open("Result/TemporalEnsembling_CIFAR10.txt", "w")

model=TemporalEnsembling(lambda_u=30,ema_weight=0.6,warmup=0.4,mu=1,weight_decay=5e-4,epoch=1,num_it_epoch=10,
                         num_it_total=540*10,eval_epoch=10,device='cpu',
                         labeled_dataset=labeled_dataset,
                         unlabeled_dataset=unlabeled_dataset,
                         valid_dataset=valid_dataset,
                         test_dataset=test_dataset,
                         labeled_sampler=labeled_sampler,
                         unlabeled_sampler=unlabeled_sampler,
                         valid_sampler=valid_sampler,
                         test_sampler=test_sampler,
                         labeled_dataloader=labeled_dataloader,
                         unlabeled_dataloader=unlabeled_dataloader,
                         valid_dataloader=valid_dataloader,
                         test_dataloader=test_dataloader,
                         augmentation=augmentation,
                         network=network,
                         optimizer=optimizer,
                         scheduler=scheduler,
                         evaluation=evaluation,
                         file=file, verbose=True
                         )

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X,valid_X=valid_X,valid_y=valid_y)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

1
2
3
4
5
6
7
8
9
10
accuracy   0.098
top_5_accuracy   0.4996
precision   0.022668997668997666
Recall   0.098
F1   0.018852372164484196


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.4748847111111112
Confusion_matrix   [[0.    0.018 0.    0.    0.    0.    0.032 0.    0.948 0.002]
 [0.    0.005 0.    0.    0.    0.    0.    0.    0.995 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.007 0.    0.991 0.001]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.003 0.    0.    0.    0.    0.001 0.    0.996 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.002 0.    0.    0.    0.    0.    0.    0.998 0.   ]
 [0.    0.007 0.    0.    0.    0.    0.017 0.    0.975 0.001]
 [0.    0.    0.    0.    0.    0.    0.    0.    1.    0.   ]]
accuracy   0.098
top_5_accuracy   0.4996
precision   0.022668997668997666
Recall   0.098
F1   0.018852372164484196


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.4748847111111112
Confusion_matrix   [[0.    0.018 0.    0.    0.    0.    0.032 0.    0.948 0.002]
 [0.    0.005 0.    0.    0.    0.    0.    0.    0.995 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.007 0.    0.991 0.001]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.003 0.    0.    0.    0.    0.001 0.    0.996 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.001 0.    0.    0.    0.    0.    0.    0.999 0.   ]
 [0.    0.002 0.    0.    0.    0.    0.    0.    0.998 0.   ]
 [0.    0.007 0.    0.    0.    0.    0.017 0.    0.975 0.001]
 [0.    0.    0.    0.    0.    0.    0.    0.    1.    0.   ]]


{'accuracy': 0.098,
 'top_5_accuracy': 0.4996,
 'precision': 0.022668997668997666,
 'Recall': 0.098,
 'F1': 0.018852372164484196,
 'AUC': 0.4748847111111112,
 'Confusion_matrix': array([[0.   , 0.018, 0.   , 0.   , 0.   , 0.   , 0.032, 0.   , 0.948,
         0.002],
        [0.   , 0.005, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.995,
         0.   ],
        [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.007, 0.   , 0.991,
         0.001],
        [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.999,
         0.   ],
        [0.   , 0.003, 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.996,
         0.   ],
        [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.999,
         0.   ],
        [0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.999,
         0.   ],
        [0.   , 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.998,
         0.   ],
        [0.   , 0.007, 0.   , 0.   , 0.   , 0.   , 0.017, 0.   , 0.975,
         0.001],
        [0.   , 0.   

## <font color='green'>Mean teacher</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Perturbation-based -> <font color='green'>Mean teacher</font>

### Opis

In [16]:
from LAMDA_SSL.Augmentation.Vision.RandomHorizontalFlip import RandomHorizontalFlip
from LAMDA_SSL.Augmentation.Vision.RandomCrop import RandomCrop
from LAMDA_SSL.Dataset.Vision.CIFAR10 import CIFAR10
from LAMDA_SSL.Opitimizer.SGD import SGD
from LAMDA_SSL.Scheduler.CosineAnnealingLR import CosineAnnealingLR
from LAMDA_SSL.Network.WideResNet import WideResNet
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader
from LAMDA_SSL.Algorithm.Classification.MeanTeacher import MeanTeacher
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler
from sklearn.pipeline import Pipeline
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset

# dataset
dataset=CIFAR10(root='Download\cifar-10-python',labeled_size=4000,stratified=True,shuffle=True,download=False,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y

unlabeled_X=dataset.unlabeled_X

test_X=dataset.test_X
test_y=dataset.test_y

valid_X=dataset.valid_X
valid_y=dataset.valid_y

labeled_dataset=LabeledDataset(pre_transform=dataset.pre_transform,transforms=dataset.transforms,
                               transform=dataset.transform,target_transform=dataset.target_transform)

unlabeled_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.unlabeled_transform)

valid_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.valid_transform)

test_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.test_transform)

# sampler
labeled_sampler=RandomSampler(replacement=True,num_samples=64*(2**20))
unlabeled_sampler=RandomSampler(replacement=True)
valid_sampler=SequentialSampler()
test_sampler=SequentialSampler()

#dataloader
labeled_dataloader=LabeledDataLoader(batch_size=64,num_workers=0,drop_last=True)
unlabeled_dataloader=UnlabeledDataLoader(num_workers=0,drop_last=True)
valid_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)
test_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)

# augmentation
augmentation=Pipeline([('RandomHorizontalFlip',RandomHorizontalFlip()),
                        ('RandomCrop',RandomCrop(padding=0.125,padding_mode='reflect')),
                      ])

# optimizer
optimizer=SGD(lr=0.03,momentum=0.9,nesterov=True)

# scheduler
scheduler=CosineAnnealingLR(eta_min=0,T_max=2**20)

# network
network=WideResNet(num_classes=10,depth=28,widen_factor=2,drop_rate=0)

# evalutation
evaluation={
    'accuracy':Accuracy(),
    'top_5_accuracy':Top_k_Accurary(k=5),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

file = open("Result/MeanTeacher_CIFAR10.txt", "w")

model=MeanTeacher(lambda_u=50,warmup=0.4,mu=1,weight_decay=5e-4,ema_decay=0.999,
                  epoch=1, num_it_epoch=10, num_it_total=540*10, eval_it=2000, device='cpu',
                  labeled_dataset=labeled_dataset,
                  unlabeled_dataset=unlabeled_dataset,
                  valid_dataset=valid_dataset,
                  test_dataset=test_dataset,
                  labeled_sampler=labeled_sampler,
                  unlabeled_sampler=unlabeled_sampler,
                  valid_sampler=valid_sampler,
                  test_sampler=test_sampler,
                  labeled_dataloader=labeled_dataloader,
                  unlabeled_dataloader=unlabeled_dataloader,
                  valid_dataloader=valid_dataloader,
                  test_dataloader=test_dataloader,
                  augmentation=augmentation,
                  network=network,
                  optimizer=optimizer,
                  scheduler=scheduler,
                  evaluation=evaluation,
                  file=file,
                  verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X,valid_X=valid_X,valid_y=valid_y)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

1
2
3
4
5
6
7
8
9
10
accuracy   0.1
top_5_accuracy   0.4983
precision   0.01
Recall   0.1
F1   0.01818181818181818


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.5442325333333334
Confusion_matrix   [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
accuracy   0.1
top_5_accuracy   0.4983
precision   0.01
Recall   0.1
F1   0.01818181818181818


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.5442325333333334
Confusion_matrix   [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


{'accuracy': 0.1,
 'top_5_accuracy': 0.4983,
 'precision': 0.01,
 'Recall': 0.1,
 'F1': 0.01818181818181818,
 'AUC': 0.5442325333333334,
 'Confusion_matrix': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])}

## <font color='green'>VAT Virtual Adversarial Training</font> LAMDA-SSL

###  Semi-supervised classification -> Intrinsically semi-supervised -> Perturbation-based -> <font color='green'>Virtual Adversarial Training</font>

### Opis

In [18]:
from LAMDA_SSL.Augmentation.Vision.RandomHorizontalFlip import RandomHorizontalFlip
from LAMDA_SSL.Augmentation.Vision.RandomCrop import RandomCrop
from LAMDA_SSL.Dataset.Vision.CIFAR10 import CIFAR10
from LAMDA_SSL.Opitimizer.SGD import SGD
from LAMDA_SSL.Scheduler.CosineAnnealingLR import CosineAnnealingLR
from LAMDA_SSL.Network.WideResNet import WideResNet
from LAMDA_SSL.Dataloader.UnlabeledDataloader import UnlabeledDataLoader
from LAMDA_SSL.Dataloader.LabeledDataloader import LabeledDataLoader
from LAMDA_SSL.Algorithm.Classification.VAT import VAT
from LAMDA_SSL.Sampler.RandomSampler import RandomSampler
from LAMDA_SSL.Sampler.SequentialSampler import SequentialSampler
from sklearn.pipeline import Pipeline
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Evaluation.Classifier.Top_k_Accuracy import Top_k_Accurary
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
from LAMDA_SSL.Dataset.LabeledDataset import LabeledDataset
from LAMDA_SSL.Dataset.UnlabeledDataset import UnlabeledDataset

# dataset
dataset=CIFAR10(root='Download\cifar-10-python',labeled_size=4000,stratified=True,shuffle=True,download=False,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y

unlabeled_X=dataset.unlabeled_X

test_X=dataset.test_X
test_y=dataset.test_y

valid_X=dataset.valid_X
valid_y=dataset.valid_y

labeled_dataset=LabeledDataset(pre_transform=dataset.pre_transform,transforms=dataset.transforms,
                               transform=dataset.transform,target_transform=dataset.target_transform)

unlabeled_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.unlabeled_transform)

valid_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.valid_transform)

test_dataset=UnlabeledDataset(pre_transform=dataset.pre_transform,transform=dataset.test_transform)

# sampler
labeled_sampler=RandomSampler(replacement=True,num_samples=64*(2**20))
unlabeled_sampler=RandomSampler(replacement=True)
valid_sampler=SequentialSampler()
test_sampler=SequentialSampler()

#dataloader
labeled_dataloader=LabeledDataLoader(batch_size=64,num_workers=0,drop_last=True)
unlabeled_dataloader=UnlabeledDataLoader(num_workers=0,drop_last=True)
valid_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)
test_dataloader=UnlabeledDataLoader(batch_size=64,num_workers=0,drop_last=False)

# augmentation

augmentation=Pipeline([('RandomHorizontalFlip',RandomHorizontalFlip()),
                        ('RandomCrop',RandomCrop(padding=0.125,padding_mode='reflect')),
                      ])

# optimizer
optimizer=SGD(lr=0.03,momentum=0.9,nesterov=True)

# scheduler
scheduler=CosineAnnealingLR(eta_min=0,T_max=2**20)

# network
network=WideResNet(num_classes=10,depth=28,widen_factor=2,drop_rate=0)

# evalutation
evaluation={
    'accuracy':Accuracy(),
    'top_5_accuracy':Top_k_Accurary(k=5),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

file = open("Result/VAT_CIFAR10.txt", "w")

model=VAT(lambda_u=0.3,lambda_entmin=0.06,eps=6,xi=1e-6,it_vat=1,warmup=0.4,mu=1,
          weight_decay=5e-4, ema_decay=0.999,
          epoch=1, num_it_epoch=10, num_it_total=540*10,
          eval_it=2000, device='cpu',
          labeled_dataset=labeled_dataset,
          unlabeled_dataset=unlabeled_dataset,
          valid_dataset=valid_dataset,
          test_dataset=test_dataset,
          labeled_sampler=labeled_sampler,
          unlabeled_sampler=unlabeled_sampler,
          valid_sampler=valid_sampler,
          test_sampler=test_sampler,
          labeled_dataloader=labeled_dataloader,
          unlabeled_dataloader=unlabeled_dataloader,
          valid_dataloader=valid_dataloader,
          test_dataloader=test_dataloader,
          augmentation=augmentation,
          network=network,
          optimizer=optimizer,
          scheduler=scheduler,
          evaluation=evaluation,
          file=file, verbose=True
          )

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X,valid_X=valid_X,valid_y=valid_y)

performance=model.evaluate(X=test_X,y=test_y)

result=model.y_pred

performance

1
2
3
4
5
6
7
8
9
10
accuracy   0.1
top_5_accuracy   0.4656
precision   0.01
Recall   0.1
F1   0.01818181818181818


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.4791982777777777
Confusion_matrix   [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
accuracy   0.1
top_5_accuracy   0.4656
precision   0.01
Recall   0.1
F1   0.01818181818181818


c:\users\kaami\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC   0.4791982777777777
Confusion_matrix   [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


{'accuracy': 0.1,
 'top_5_accuracy': 0.4656,
 'precision': 0.01,
 'Recall': 0.1,
 'F1': 0.01818181818181818,
 'AUC': 0.4791982777777777,
 'Confusion_matrix': array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])}

# Transductive

## <font color='green'>TSVM</font> LAMDA-SSL

###  Semi-supervised classification -> Transductive -> <font color='green'>TSVM</font>

### Opis

[LAMDA-SSl]

1. Rozpoczęcie od tradycyjnego SVM: TSVM zaczyna od wytrenowania standardowego SVM wykorzystującego oznaczone próbki danych. Hiperpłaszczyzna uzyskana z tego kroku jest używana jako punkt wyjścia do dalszej optymalizacji.

2. Próba etykietowania próbek nieoznaczonych: Używając wytrenowanego SVM, TSVM przewiduje etykiety dla próbek nieoznaczonych. Należy jednak pamiętać, że te etykiety są początkowymi, przybliżonymi przewidywaniami i mogą się zmieniać w kolejnych krokach.

3. Optymalizacja hiperpłaszczyzny: TSVM dąży do optymalizacji hiperpłaszczyzny dzielącej tak, aby zminimalizować błędy klasyfikacji zarówno dla próbek oznaczonych, jak i dla tych początkowo etykietowanych próbek nieoznaczonych. Proces ten może być realizowany przez iteracyjną procedurę, gdzie w każdej iteracji TSVM aktualizuje etykiety dla próbek nieoznaczonych oraz dostosowuje hiperpłaszczyznę dzielącą.

4. Kary i ważność próbek: Ważność przywiązywana do błędów w klasyfikacji różni się dla próbek oznaczonych i nieoznaczonych. Dlatego TSVM wprowadza różne kary (lub wagi) dla tych dwóch rodzajów próbek. W trakcie iteracji, ważność próbek nieoznaczonych jest dostosowywana, co może prowadzić do zmiany ich etykietowania.

5. Zakończenie procesu: Proces iteracyjny jest kontynuowany do momentu, gdy spełnione są pewne kryteria zbieżności (na przykład gdy liczba błędnie etykietowanych próbek nieoznaczonych nie zmienia się między kolejnymi iteracjami lub osiągnie się określony próg).

In [5]:
from LAMDA_SSL.Algorithm.Classification.TSVM import TSVM
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
import numpy as np

file = open("Result/TSVM_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}
model=TSVM(evaluation=evaluation,file=file)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y,Transductive=False)

result=model.y_pred

performance

{'accuracy': 0.9590643274853801,
 'precision': 0.9647397094430993,
 'Recall': 0.9484521028037383,
 'F1': 0.9555629802873371,
 'AUC': 0.9941588785046729,
 'Confusion_matrix': array([[0.90625   , 0.09375   ],
        [0.00934579, 0.99065421]])}

# <font color='blue'>Graph-based</font>

## <font color='green'>Label propagation algorithm for inference on graphs</font> <font color='orange'> SKLEARN</font> or LAMDA-SSL

###  Semi-supervised classification -> Transductive -> Graph-based -> Transductive -> Inference in graphs -> <font color='green'>Label propagation</font>

### Opis

[LAMDA-SSL]

Label Propagation to metoda uczenia półnadzorowanego zaproponowana przez Zhu i innych. Wykorzystuje ona próbki jako węzły oraz ich relacje jako krawędzie w grafie. Graf ten może być w pełni połączony lub oparty na k-najbliższych sąsiadach. Głównym celem algorytmu Label Propagation jest przekazywanie etykiet z oznaczonych danych do nieoznaczonych przez graf. Optymalizacja polega na zapewnieniu spójności Laplace'a, czyli minimalizacji ważonego błędu średniokwadratowego różnic etykiet między sąsiadującymi węzłami. Ponieważ etykiety próbek oznaczonych są stałe, algorytm skupia się na etykietowaniu danych nieoznaczonych. Metoda ta oferuje optymalne rozwiązanie w postaci zamkniętej, które jest zbieżne z wynikiem nieskończonych iteracji. Jedną z głównych zalet Label Propagation w porównaniu z innymi metodami półnadzorowanymi opartymi na grafie jest jej zdolność do uzyskiwania dokładnych rozwiązań bez potrzeby wielokrotnej symulacji procesu propagacji etykiet.

[A survey on semi-supervised learning]

Algorytm Label Propagation, zaproponowany przez Zhu i Ghahramani, służy do wnioskowania na grafach. W tym podejściu etykiety są przekazywane (propagowane) z węzła do węzła na podstawie wag krawędzi. Każda estymowana etykieta w węźle jest obliczana jako ważona suma etykiet sąsiednich węzłów. W notacji macierzowej, istnieje macierz przejścia opisująca te ważenia. Algorytm składa się z dwóch kroków: propagacji etykiet do sąsiadujących węzłów i resetowania przewidywań dla oznaczonych punktów danych do ich prawdziwych etykiet. Procedura jest powtarzana do momentu zbieżności.

Zhu w 2005 roku udowodnił, że algorytm zbiega do funkcji harmonicznego rozwiązania. Pokazał również, że Label Propagation może być interpretowany jako losowy spacer z macierzą przejścia, który zatrzymuje się, gdy osiągnie węzeł z etykietą. Ten sposób patrzenia jest zbliżony do podejścia opartego na Markovskich losowych spacerach, gdzie istnieje prawdopodobieństwo zatrzymania spaceru, gdy osiągnie węzeł z etykietą. Istnieją również prace innych badaczy, które przedstawiają podobne podejścia lub rozszerzenia algorytmu Label Propagation.

In [10]:
from LAMDA_SSL.Algorithm.Classification.LabelPropagation import LabelPropagation
from LAMDA_SSL.Evaluation.Classifier.Accuracy import Accuracy
from LAMDA_SSL.Dataset.Tabular.BreastCancer import BreastCancer
from LAMDA_SSL.Evaluation.Classifier.Recall import Recall
from LAMDA_SSL.Evaluation.Classifier.F1 import F1
from LAMDA_SSL.Evaluation.Classifier.Precision import Precision
from LAMDA_SSL.Evaluation.Classifier.AUC import AUC
from LAMDA_SSL.Evaluation.Classifier.Confusion_Matrix import Confusion_Matrix
import numpy as np

file = open("Result/LabelPropagation_BreastCancer.txt", "w")

dataset=BreastCancer(test_size=0.3,labeled_size=0.1,stratified=True,shuffle=True,random_state=0,default_transforms=True)

labeled_X=dataset.labeled_X
labeled_y=dataset.labeled_y
unlabeled_X=dataset.unlabeled_X
unlabeled_y=dataset.unlabeled_y
test_X=dataset.test_X
test_y=dataset.test_y

# Pre_transform
pre_transform=dataset.pre_transform
pre_transform.fit(np.vstack([labeled_X, unlabeled_X]))

labeled_X=pre_transform.transform(labeled_X)
unlabeled_X=pre_transform.transform(unlabeled_X)
test_X=pre_transform.transform(test_X)

evaluation={
    'accuracy':Accuracy(),
    'precision':Precision(average='macro'),
    'Recall':Recall(average='macro'),
    'F1':F1(average='macro'),
    'AUC':AUC(multi_class='ovo'),
    'Confusion_matrix':Confusion_Matrix(normalize='true')
}

model=LabelPropagation(gamma=1,max_iter=10000,evaluation=evaluation,file=file,verbose=True)

model.fit(X=labeled_X,y=labeled_y,unlabeled_X=unlabeled_X)

performance=model.evaluate(X=test_X,y=test_y,Transductive=False)

result=model.y_pred

performance

{'accuracy': 0.9532163742690059,
 'precision': 0.9652173913043478,
 'Recall': 0.9375,
 'F1': 0.9486486486486486,
 'AUC': 0.9976635514018691,
 'Confusion_matrix': array([[0.875, 0.125],
        [0.   , 1.   ]])}